<a href="https://colab.research.google.com/github/sledhead/simple_python/blob/main/webpage_scrapv4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Required Modules (Libraries)

*   Link to Google Drive
*   Install Modules to interact with xml
*   Link Required libraries

In [1]:
import pandas as pd
import requests
#Will display upload box to user to allow a file to upload and encrypted.
from google.colab import drive
drive.mount('/content/drive')

!pip install xmltodict
import xmltodict

import time
import os
import shutil

Mounted at /content/drive


# Senator Vote Records


*   Get the senator vote records for each session
*   Convert the records into individual vote tallies
*   Provide User with ability to control time period

Section below will gather xml file for each year of the senate vote.... 2017 to current

In [2]:
#allow the user the choice to define the begin years and end years....
#https://www.senate.gov/legislative/votes_new.htm
#build a list from the site above....
#will start with 1990 and work forward to current day
#1990 = vote_menu_101_1.xml

def combine_nums(cong_id,sess_id):
  combine_str = str(cong_id) + '_' + str(sess_id)
  return combine_str

year_lst = range(1989,2025,1)
congress_id_lst = range(101,119,1)
session_lst = [1,2]

congress_session_lst = [ [combine_nums(congress_id_num, session_id) for session_id in session_lst ] for congress_id_num in congress_id_lst ]
flatten_congress_session_lst = [ final_out for pair in congress_session_lst for final_out in pair ]
#flatten_list = [j for sub in ini_list for j in sub]
#zip()

combine_congress_year = zip(year_lst,flatten_congress_session_lst)
vote_menu_dict = {}

for year_num in combine_congress_year:
  vote_menu_dict[year_num[0]] = 'vote_menu_' + year_num[1] + '.xml'


In [3]:
# @title Please Select the Year Range  Min to Max { display-mode: "form" }

min_year = '2017' # @param ['1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']
max_year = '2024' # @param ['1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']

#build list of full file path required by the user:
#take the min and max and build a range of year required
year_num_req = range(int(min_year),(int(max_year)+1),1)
base_web_link = 'https://www.senate.gov/legislative/LIS/roll_call_lists/'

web_link_lst = []
for each_yr in year_num_req:
  file_name_part = vote_menu_dict[each_yr]
  complete_path = base_web_link + file_name_part
  web_link_lst.append(complete_path)


#now get the requested files from website....

for single_web_link in web_link_lst:
  !wget { single_web_link }

--2024-07-31 03:30:46--  https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_115_1.xml
Resolving www.senate.gov (www.senate.gov)... 184.26.238.102, 2600:1407:7400:186::1e3, 2600:1407:7400:18a::1e3
Connecting to www.senate.gov (www.senate.gov)|184.26.238.102|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/xml]
Saving to: ‘vote_menu_115_1.xml’

vote_menu_115_1.xml     [ <=>                ] 149.40K  --.-KB/s    in 0.08s   

2024-07-31 03:30:46 (1.76 MB/s) - ‘vote_menu_115_1.xml’ saved [152990]

--2024-07-31 03:30:46--  https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_115_2.xml
Resolving www.senate.gov (www.senate.gov)... 184.26.238.102, 2600:1407:7400:186::1e3, 2600:1407:7400:18a::1e3
Connecting to www.senate.gov (www.senate.gov)|184.26.238.102|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/xml]
Saving to: ‘vote_menu_115_2.xml’

vote_menu_115_2.xml     [ <=>              

Next Section below will open xml and then use this file to build the list of file links for gather senator votes

In [4]:
#Base file path for local folder struct in vm
current_dir = os.getcwd()
local_base_folder_path = os.path.join(current_dir,'Senator_data')
if( os.path.exists(local_base_folder_path) == False ):
  os.makedirs(local_base_folder_path)

congress_session_folder_names = []
full_http_link_file_path = []
each_vote_info = []
#vote_menu_file_lst = ['vote_menu_115_1.xml','vote_menu_115_2.xml','vote_menu_116_1.xml','vote_menu_116_2.xml','vote_menu_117_1.xml','vote_menu_117_2.xml','vote_menu_118_1.xml','vote_menu_118_2.xml']

#for each_year_call_lst in vote_menu_file_lst:
for each_year in year_num_req:
  file_name_part = vote_menu_dict[each_year]

  xml_file = open(file_name_part, 'r')
  xml_data_str = xml_file.read()

  xml_to_dict_obj = xmltodict.parse(xml_data_str)

  congress_num = xml_to_dict_obj['vote_summary']['congress']
  congress_sec_num = xml_to_dict_obj['vote_summary']['session']

  congress_lst_vote = xml_to_dict_obj['vote_summary']['votes']['vote']



  base_link_path = 'https://www.senate.gov/legislative/LIS/roll_call_votes/'
  full_base_link_path = base_link_path + 'vote' + str(congress_num) + str(congress_sec_num) + '/vote_' + str(congress_num) + '_' + str(congress_sec_num) + '_'
  congress_session_folder_names.append('vote_' + str(congress_num) + '_' + str(congress_sec_num))
  print(full_base_link_path)

  for each_vote_section in congress_lst_vote:
    single_vote_num = each_vote_section['vote_number']
    single_vote_info_dict = {}
    #build full link path for getting xml data....
    #https://www.senate.gov/legislative/LIS/roll_call_votes/vote1172/vote_117_2_00420.htm

    single_file_link_path = full_base_link_path + str(single_vote_num) + '.xml'
    full_http_link_file_path.append(single_file_link_path)
    single_vote_info_dict['full_http_path'] = single_file_link_path
    single_vote_info_dict['dest_folder'] = 'vote_' + str(congress_num) + '_' + str(congress_sec_num)
    single_vote_info_dict['file_name'] = 'vote_' + str(congress_num) + '_' + str(congress_sec_num) + '_' + str(single_vote_num) + '.xml'
    each_vote_info.append(single_vote_info_dict)


#section will build the temp folder directory struct for this vm instanse
print('..............................................................')
for new_folder_name in congress_session_folder_names:
  #build new folder path and make
  local_folder_path = os.path.join(current_dir,'Senator_data',new_folder_name)
  if( os.path.exists(local_folder_path) == False ):
    print(f'The following folder was added: {local_folder_path}')
    os.makedirs(local_folder_path)

https://www.senate.gov/legislative/LIS/roll_call_votes/vote1151/vote_115_1_
https://www.senate.gov/legislative/LIS/roll_call_votes/vote1152/vote_115_2_
https://www.senate.gov/legislative/LIS/roll_call_votes/vote1161/vote_116_1_
https://www.senate.gov/legislative/LIS/roll_call_votes/vote1162/vote_116_2_
https://www.senate.gov/legislative/LIS/roll_call_votes/vote1171/vote_117_1_
https://www.senate.gov/legislative/LIS/roll_call_votes/vote1172/vote_117_2_
https://www.senate.gov/legislative/LIS/roll_call_votes/vote1181/vote_118_1_
https://www.senate.gov/legislative/LIS/roll_call_votes/vote1182/vote_118_2_
..............................................................
The following folder was added: /content/Senator_data/vote_115_1
The following folder was added: /content/Senator_data/vote_115_2
The following folder was added: /content/Senator_data/vote_116_1
The following folder was added: /content/Senator_data/vote_116_2
The following folder was added: /content/Senator_data/vote_117_1
The 

In [5]:
# @title Please select local file storage only or local plus drive backup { display-mode: "form" }

location = 'drive_backup' # @param ['local_only','drive_backup']
backup_path = '/content/drive/MyDrive/Encryption_test/Senator_data' # @param {type:"string"}


for single_vote_xml_file in each_vote_info:
  #get the request xml data files
  #get the xml data files
  if( location == 'local_only' ):
    #just store xml file on the local vm....
    local_dest_folder_path = os.path.join(current_dir,'Senator_data',single_vote_xml_file['dest_folder'])
    !wget -P {local_dest_folder_path} {single_vote_xml_file['full_http_path']}

  if( location == 'drive_backup'):
    #more stuff to do...
    #does drive path exists???
    local_dest_folder_path = os.path.join(backup_path,single_vote_xml_file['dest_folder'])
    if( os.path.exists(local_dest_folder_path) == False ):
      os.makedirs(local_dest_folder_path)

    #before download a new copy, check and see if it exists on the drive backup...
    check_for_xml_file = os.path.join(backup_path,single_vote_xml_file['dest_folder'], single_vote_xml_file['file_name'])
    if( os.path.exists(check_for_xml_file) == False ):
      #does not exist in back_up, so download
      !wget -P {local_dest_folder_path} {single_vote_xml_file['full_http_path']}

    else:
      #file was found...
      print(f'Found:::::: {check_for_xml_file}')

Found:::::: /content/drive/MyDrive/Encryption_test/Senator_data/vote_115_1/vote_115_1_00325.xml
Found:::::: /content/drive/MyDrive/Encryption_test/Senator_data/vote_115_1/vote_115_1_00324.xml
Found:::::: /content/drive/MyDrive/Encryption_test/Senator_data/vote_115_1/vote_115_1_00323.xml
Found:::::: /content/drive/MyDrive/Encryption_test/Senator_data/vote_115_1/vote_115_1_00322.xml
Found:::::: /content/drive/MyDrive/Encryption_test/Senator_data/vote_115_1/vote_115_1_00321.xml
Found:::::: /content/drive/MyDrive/Encryption_test/Senator_data/vote_115_1/vote_115_1_00320.xml
Found:::::: /content/drive/MyDrive/Encryption_test/Senator_data/vote_115_1/vote_115_1_00319.xml
Found:::::: /content/drive/MyDrive/Encryption_test/Senator_data/vote_115_1/vote_115_1_00318.xml
Found:::::: /content/drive/MyDrive/Encryption_test/Senator_data/vote_115_1/vote_115_1_00317.xml
Found:::::: /content/drive/MyDrive/Encryption_test/Senator_data/vote_115_1/vote_115_1_00316.xml
Found:::::: /content/drive/MyDrive/Encry

# Build Senator Dictionary

In [6]:
def GetVoteLayout(senator_vote_list):
  #Generate additional information about the over flow of the vote.
  #Determine the party majority of both yeas and nays.
  #{'yea_major':R or D, yea_R: cnt, yea_D: cnt, yea_I:cnt, nay_major: R or D, nay_R: cnt, nay_D: cnt, nay_I:cnt}
  vote_info = {}
  yea_D_lst = [single_senator for single_senator in senator_vote_list if( single_senator['party'] == 'D' and single_senator['vote_cast'] == 'Yea') == True]
  #print(len(yea_D_lst))

  yea_R_lst = [single_senator for single_senator in senator_vote_list if( single_senator['party'] == 'R' and single_senator['vote_cast'] == 'Yea') == True]
  #print(len(yea_R_lst))

  yea_I_lst = [single_senator for single_senator in senator_vote_list if( single_senator['party'] == 'I' and single_senator['vote_cast'] == 'Yea') == True]
  #print(len(yea_I_lst))

  #calulate which party has major....
  if(len(yea_D_lst) > len(yea_R_lst)):
    vote_info['yea_major'] = 'D'
  elif(len(yea_D_lst) < len(yea_R_lst)):
    vote_info['yea_major'] = 'R'
  elif(len(yea_D_lst) == len(yea_R_lst)):
    vote_info['yea_major'] = 'E'

  vote_info['yea_D'] = len(yea_D_lst)
  vote_info['yea_R'] = len(yea_R_lst)
  vote_info['yea_I'] = len(yea_I_lst)


  #print('.........................................................')

  nays_D_lst = [single_senator for single_senator in senator_vote_list if( single_senator['party'] == 'D' and single_senator['vote_cast'] == 'Nay') == True]
  #print(len(nays_D_lst))

  nays_R_lst = [single_senator for single_senator in senator_vote_list if( single_senator['party'] == 'R' and single_senator['vote_cast'] == 'Nay') == True]
  #print(len(nays_R_lst))

  nays_I_lst = [single_senator for single_senator in senator_vote_list if( single_senator['party'] == 'I' and single_senator['vote_cast'] == 'Nay') == True]
  #print(len(nays_I_lst))

  #calulate which party has major....
  if(len(nays_D_lst) > len(nays_R_lst)):
    vote_info['nay_major'] = 'D'
  elif(len(nays_D_lst) < len(nays_R_lst)):
    vote_info['nay_major'] = 'R'
  elif(len(nays_D_lst) == len(nays_R_lst)):
    vote_info['nay_major'] = 'E'

  vote_info['nay_D'] = len(nays_D_lst)
  vote_info['nay_R'] = len(nays_R_lst)
  vote_info['nay_I'] = len(nays_I_lst)

  return vote_info

Code Below will allow the user to select the Senator that will be processed for comparsion.

In [7]:
# @title Please Enter a string of Senator seperated by a comma { display-mode: "form" }

Senator_str_lst = "Daines, Tester, Cruz, Schumer, Warren, Cotton, Sanders, Menendez, Paul,Hirono" # @param {type:"string"}

#clean up the string before converting to list
clean_up_Senator_str = Senator_str_lst.replace(" ", "")
#make string into list for code below
user_defined_senator_only_lst = clean_up_Senator_str.split(sep=',')


In [29]:
#add simple function which look at the object type and return the correct value based on object type
#for what ever reason, senator website returns multiple objects for some xml files.. document object
#is listed more than one time...
def GetCorrectDocumentData(document_object):
  filter_dict_object = {}

  if( type( document_object ) == [].__class__ ):
    #found list return the first index which contains the dict needed.
    filter_dict_object = document_object[0]

  else:
    filter_dict_object = document_object

  return filter_dict_object

In [32]:
missing_links = []
senator_history_lst = []

#full_http_link_file_path = ['https://www.senate.gov/legislative/LIS/roll_call_votes/vote1151/vote_115_1_00271.xml', 'https://www.senate.gov/legislative/LIS/roll_call_votes/vote1151/vote_115_1_00262.xml']
#select_only_senators = ['Daines', 'Tester', 'Cruz', 'Schumer','Warren','Cotton','Sanders', 'Menendez','Paul','Hirono']
select_only_senators = user_defined_senator_only_lst

for each_single_file_path in each_vote_info:
  #gather the data from each single file link path
  #change this code... the xml have been downloaded to local or backup
  #r = requests.get(each_single_file_link)
  #print(r.status_code)
  #print(r.text)

  #make sure we got a file....
  #depend what the user has selected for file location... local vm or backup on google drive
  if( location == 'local_only' ):
    #set base folder path to local vm...
    base_vote_folder_path =  os.path.join(current_dir,'Senator_data',each_single_file_path['dest_folder'])

  elif( location == 'drive_backup' ):
    #get files from the backup location
    base_vote_folder_path = os.path.join(backup_path,each_single_file_path['dest_folder'])

  complete_vote_file_path = os.path.join(base_vote_folder_path,each_single_file_path['file_name'])

  if(os.path.exists(complete_vote_file_path) == True ):
    vote_call_details = {}
    with open( complete_vote_file_path, 'r') as fd:
      vote_call_details = xmltodict.parse(fd.read())

    #vote_call_details = xmltodict.parse(r.text)

    congress_id_num = vote_call_details['roll_call_vote']['congress']
    congress_session_num = vote_call_details['roll_call_vote']['session']
    vote_call_num = vote_call_details['roll_call_vote']['vote_number']
    vote_call_issue = vote_call_details['roll_call_vote']['vote_title']
    majority_req = vote_call_details['roll_call_vote']['majority_requirement']
    #print(type(vote_call_details['roll_call_vote']['document']))
    #document_type = vote_call_details['roll_call_vote']['document']['document_type']
    #document_number = vote_call_details['roll_call_vote']['document']['document_number']


    if( 'document' in vote_call_details['roll_call_vote'] ):
      document_type = GetCorrectDocumentData(vote_call_details['roll_call_vote']['document'])['document_type']
      document_number = GetCorrectDocumentData(vote_call_details['roll_call_vote']['document'])['document_number']

    else:
      #does not exist
      document_type = 'N/A'
      document_number = 'N/A'

    #setup loop for what each senator voted...
    senator_member_lst = vote_call_details['roll_call_vote']['members']['member']
    select_senator_vote_dict = {dict['last_name']:dict['vote_cast'] for dict in senator_member_lst if (dict['last_name'] in select_only_senators) == True }
    print(f'Currently working on: Congress: {congress_id_num}, session: {congress_session_num}, vote call: {vote_call_num}')

    new_row_dict = {}
    #add an additional field majority_requirement
    #would allow code to determine what percentage is required for majority
    new_row_dict['congress'] = congress_id_num
    new_row_dict['session'] = congress_session_num
    new_row_dict['vote_number'] = vote_call_num
    new_row_dict['vote_title'] = vote_call_issue
    new_row_dict['majority_requirement'] = majority_req
    new_row_dict['document_type'] = document_type
    new_row_dict['document_number'] = document_number

    #add section to keep track of yea and nay counts.
    new_row_dict['yea_cnt'] = vote_call_details['roll_call_vote']['count']['yeas']
    new_row_dict['nays_cnt'] = vote_call_details['roll_call_vote']['count']['nays']
    new_row_dict['vote_deep_dive'] = GetVoteLayout(senator_member_lst)

    for each_sen_record in select_only_senators:
      #print(each_sen_mem['last_name'])

      new_row_dict[each_sen_record] = select_senator_vote_dict[each_sen_record]

      #add to list for later examation
      #after building list, will add to dataframe.

    senator_history_lst.append(new_row_dict)

  else:
    #something went wrong...
    #keep track of this problem
    missing_links.append(complete_vote_file_path)

  #before moving to the next record, pause for some time....
  #time.sleep(0.25)

Currently working on: Congress: 115, session: 1, vote call: 325
Currently working on: Congress: 115, session: 1, vote call: 324
Currently working on: Congress: 115, session: 1, vote call: 323
Currently working on: Congress: 115, session: 1, vote call: 322
Currently working on: Congress: 115, session: 1, vote call: 321
Currently working on: Congress: 115, session: 1, vote call: 320
Currently working on: Congress: 115, session: 1, vote call: 319
Currently working on: Congress: 115, session: 1, vote call: 318
Currently working on: Congress: 115, session: 1, vote call: 317
Currently working on: Congress: 115, session: 1, vote call: 316
Currently working on: Congress: 115, session: 1, vote call: 315
Currently working on: Congress: 115, session: 1, vote call: 314
Currently working on: Congress: 115, session: 1, vote call: 313
Currently working on: Congress: 115, session: 1, vote call: 312
Currently working on: Congress: 115, session: 1, vote call: 311
Currently working on: Congress: 115, ses

# Test Code Section


In [31]:
print(complete_vote_file_path)

vote_call_details = {}
with open( complete_vote_file_path, 'r') as fd:
  vote_call_details = xmltodict.parse(fd.read())

print(vote_call_details)

/content/drive/MyDrive/Encryption_test/Senator_data/vote_116_2/vote_116_2_00027.xml
{'roll_call_vote': {'congress': '116', 'session': '2', 'congress_year': '2020', 'vote_number': '27', 'vote_date': 'January 31, 2020,  05:35 PM', 'modify_date': 'January 31, 2020,  05:58 PM', 'vote_question_text': 'On the Motion (Shall It Be In Order To Consider And Debate Any Motion To Subpoena Witnesses Or Documents)', 'vote_document_text': None, 'vote_result_text': 'Motion Rejected (49-51)', 'question': 'On the Motion', 'vote_title': 'Shall It Be In Order To Consider And Debate Any Motion To Subpoena Witnesses Or Documents', 'majority_requirement': '1/2', 'vote_result': 'Motion Rejected', 'count': {'yeas': '49', 'nays': '51', 'present': None, 'absent': None}, 'tie_breaker': {'by_whom': None, 'tie_breaker_vote': None}, 'members': {'member': [{'member_full': 'Alexander (R-TN)', 'last_name': 'Alexander', 'first_name': 'Lamar', 'party': 'R', 'state': 'TN', 'vote_cast': 'Nay', 'lis_member_id': 'S289'}, {'m

In [16]:
print(complete_vote_file_path)
complete_vote_file_path2 = '/content/drive/MyDrive/Encryption_test/Senator_data/vote_115_1/vote_115_1_00092.xml'

vote_call_details = {}
with open( complete_vote_file_path2, 'r') as fd:
  vote_call_details = xmltodict.parse(fd.read())

print(vote_call_details)

/content/drive/MyDrive/Encryption_test/Senator_data/vote_115_1/vote_115_1_00091.xml
{'roll_call_vote': {'congress': '115', 'session': '1', 'congress_year': '2017', 'vote_number': '92', 'vote_date': 'March 21, 2017,  06:11 PM', 'modify_date': 'March 21, 2017,  07:05 PM', 'vote_question_text': 'On the Joint Resolution H.J.Res. 69', 'vote_document_text': 'A joint resolution providing for congressional disapproval under chapter 8 of title 5, United States Code, of the final rule of the Department of the Interior relating to "Non-Subsistence Take of Wildlife, and Public Participation and Closure Procedures, on National Wildlife Refuges in Alaska".', 'vote_result_text': 'Joint Resolution Passed (52-47)', 'question': 'On the Joint Resolution', 'vote_title': 'H.J. Res. 69', 'majority_requirement': '1/2', 'vote_result': 'Joint Resolution Passed', 'document': {'document_congress': '115', 'document_type': 'H.J.Res.', 'document_number': '69', 'document_name': 'H.J.Res. 69', 'document_title': 'A jo

In [23]:
#test_lst = ['me','sun','light']
test_lst = 'wee'
print(type(test_lst))
#test_dict = 'hello'
test_dict = {'one':1,'two':2}

if( type(test_lst) == [].__class__ ):
  print('found')

if( type( test_dict) == {}.__class__):
  print('the awesome one')

<class 'str'>
the awesome one


In [19]:
[].__class__

list

In [33]:
print(len(senator_history_lst))

2843
